<a href="https://colab.research.google.com/github/Difesoares/DataPrep/blob/main/DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neste notebook a ideia é apresentar o processo de preparação dos dados ( DataPrep ) para algorítmos de Machine Learning. Os dados foram separados entre treino e teste, mas apenas foi utilizado o conjuto de treino.

A preparação consiste em:

*   Análise de Metadados;
*   Tratamento de nulos;
*   Normalização / Padronização;
*   Análise da Cardinalidade;
*   OneHot Encoding;
*   Label Encoding.


In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [21]:
# Ler diretório Parquet
abt00 = pd.read_parquet('/content/drive/MyDrive/POD Academy/Feature_Engineering_Eng_dados/ABT1', engine='pyarrow')
abt00.shape

(1000, 58)

In [ ]:
abt00.head()

,Idade,GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,Plano,Churn,ID_CLIENTE,VL_TOT_CONSUMO,VL_MED_CONSUMO,VL_MAX_CONSUMO,...,VL_RAZ_MED_U3M_U6M_CONS_ALI,VL_RAZ_MED_U6M_U12M_CONS_ALI,VL_RAZ_MED_U3M_U6M_CONS_Livros,VL_RAZ_MED_U6M_U12M_CONS_Livros,VL_RAZ_MED_U3M_U6M_CONS_Roupas,VL_RAZ_MED_U6M_U12M_CONS_Roupas,VL_RAZ_MED_U3M_U6M_CONS_Elet,VL_RAZ_MED_U6M_U12M_CONS_Elet,PK_DAT_PROC,PK_DATREF
0,21,F,1331,1,Intermediário,1,1,1976.56,116.27,199.39,...,NaN,1.0,1.0,1.05,1.46,0.66,NaN,NaN,2024-07-29,202407
1,21,M,1160,0,Intermediário,0,2,1017.80,101.78,192.63,...,0.87,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-29,202407
2,62,M,454,1,Básico,0,3,997.81,110.87,191.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-29,202407
3,64,M,226,1,Intermediário,0,4,1009.77,126.22,199.35,...,1.00,1.0,NaN,NaN,NaN,0.97,NaN,NaN,2024-07-29,202407
4,61,M,474,1,Avançado,0,5,1755.18,117.01,170.78,...,NaN,NaN,NaN,NaN,1.00,1.00,NaN,NaN,2024-07-29,202407


In [ ]:
# Identificação dos campos da base

abt00.columns

Index(['Idade', 'GENERO', 'QT_DIAS_DESDE_INSCR', 'FL_USOU_SUPORTE', 'Plano',
       'Churn', 'ID_CLIENTE', 'VL_TOT_CONSUMO', 'VL_MED_CONSUMO',
       'VL_MAX_CONSUMO', 'VL_MIN_CONSUMO', 'VL_TOT_CONS_ESPORTES',
       'VL_MED_CONS_ESPORTES', 'VL_MAX_CONS_ESPORTES', 'VL_MIN_CONS_ESPORTES',
       'VL_MED_U3M_CONS_ESPORTES', 'VL_MED_U6M_CONS_ESPORTES',
       'VL_MED_U12M_CONS_ESPORTES', 'VL_TOT_CONS_Alimentos',
       'VL_MED_CONS_Alimentos', 'VL_MAX_CONS_Alimentos',
       'VL_MIN_CONS_Alimentos', 'VL_MED_U3M_CONS_Alimentos',
       'VL_MED_U6M_CONS_Alimentos', 'VL_MED_U12M_CONS_Alimentos',
       'VL_TOT_CONS_Livros', 'VL_MED_CONS_Livros', 'VL_MAX_CONS_Livros',
       'VL_MIN_CONS_Livros', 'VL_MED_U3M_CONS_Livros',
       'VL_MED_U6M_CONS_Livros', 'VL_MED_U12M_CONS_Livros',
       'VL_TOT_CONS_Roupas', 'VL_MED_CONS_Roupas', 'VL_MAX_CONS_Roupas',
       'VL_MIN_CONS_Roupas', 'VL_MED_U3M_CONS_Roupas',
       'VL_MED_U6M_CONS_Roupas', 'VL_MED_U12M_CONS_Roupas',
       'VL_TOT_CONS_Eletron

## **Tratamento inicial**

*   Separar uma parte dos dados (30%) para testes e validações.




In [22]:
# Dividindo o DataFrame em treino e teste
abt01, abt01_test = train_test_split(abt00, test_size=0.3, random_state=42)

abt01.shape,abt01_test.shape

((700, 58), (300, 58))

### **Separando colunas**

*   O target (**Churn**) será separado e reinserido após a conclusão das etapas de preparação.
*   As variáveis **ID_Cliente**, **PK_DAT_PROC** e **PK_DATREF** serão removidos, pois não acrescentam valor preditivo ao modelo e podem introduzir ruído.





In [23]:
# lista de variáveis para retirar dos tratamentos

lista_spec = ["ID_CLIENTE", "Churn", "PK_DAT_PROC", "PK_DATREF" ]

abt01 = abt01.drop(axis=1,columns = lista_spec)
abt01.shape

(700, 54)

## **Metadados**
Com o metadados conseguimos ver o tipo de cada variável, a quantidade e a percentagem de nulos e a cardinalidade.

In [24]:
def pod_academy_generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

metadata_df = pod_academy_generate_metadata(abt01)
metadata_df.head(100)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,VL_RAZ_MED_U6M_U12M_CONS_Elet,float64,700,100.00,0
1,VL_RAZ_MED_U3M_U6M_CONS_Elet,float64,700,100.00,0
2,VL_MED_U12M_CONS_Eletronicos,float64,700,100.00,0
3,VL_MED_U6M_CONS_Eletronicos,float64,700,100.00,0
4,VL_MED_U3M_CONS_Eletronicos,float64,700,100.00,0
5,VL_MED_U3M_CONS_Alimentos,float64,485,69.29,214
6,VL_RAZ_MED_U3M_U6M_CONS_Roupas,float64,485,69.29,33
7,VL_RAZ_MED_U3M_U6M_CONS_ALI,float64,485,69.29,27
8,VL_MED_U3M_CONS_Roupas,float64,485,69.29,213
9,VL_RAZ_MED_U3M_U6M_CONS_Livros,float64,483,69.00,29


## **Tratamento dos valores Nulos**

### **Analisando as variáveis numéricas**

*   Excluindo as variáveis com mais de 70% de nulos;
*   Preenchendo os campos nulos com a média para as variáveis com menos de 70% de nulos.



In [25]:
# Filtrando o DataFrame

missing_cutoff = 70
df_numericas_drop_nulos = metadata_df[(metadata_df['percent_nulos'] >= missing_cutoff)]
lista_drop_nulos_numericas = list(df_numericas_drop_nulos.nome_variavel.values)
lista_drop_nulos_numericas

['VL_RAZ_MED_U6M_U12M_CONS_Elet',
 'VL_RAZ_MED_U3M_U6M_CONS_Elet',
 'VL_MED_U12M_CONS_Eletronicos',
 'VL_MED_U6M_CONS_Eletronicos',
 'VL_MED_U3M_CONS_Eletronicos']

*   Essas são as variáveis com mais de 70% de nulos.






In [26]:
# retirando lista de variáveis com alto percentual de nulos

abt02 = abt01.drop(axis=1,columns=lista_drop_nulos_numericas)
abt02.shape

(700, 49)

*   Excluindo as variáveis que se destacaram com alto percentual de nulos.




In [27]:
# Substituindo nulos pela média da variável numérica

abt02 = abt02.fillna(abt02.select_dtypes(include=['float64','int32']).mean())
abt02.shape

(700, 49)

*   Nas variáveis que tem menos de 70% de nulos, preenchemos os campos nulos com a média.




In [15]:
abt02.head()

,Idade,GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,Plano,VL_TOT_CONSUMO,VL_MED_CONSUMO,VL_MAX_CONSUMO,VL_MIN_CONSUMO,VL_TOT_CONS_ESPORTES,...,VL_MAX_CONS_Eletronicos,VL_MIN_CONS_Eletronicos,VL_RAZ_MED_U3M_U6M_CONS_ESP,VL_RAZ_MED_U6M_U12M_CONS_ESP,VL_RAZ_MED_U3M_U6M_CONS_ALI,VL_RAZ_MED_U6M_U12M_CONS_ALI,VL_RAZ_MED_U3M_U6M_CONS_Livros,VL_RAZ_MED_U6M_U12M_CONS_Livros,VL_RAZ_MED_U3M_U6M_CONS_Roupas,VL_RAZ_MED_U6M_U12M_CONS_Roupas
541,67,F,997,0,Intermediário,304.84,152.42,158.61,146.23,0.00,...,0.0,0.0,1.005672,1.001086,0.998279,1.000915,1.000000,1.000000,1.001907,1.002509
440,55,F,1777,1,Intermediário,998.90,83.24,190.55,11.54,17.01,...,0.0,0.0,1.005672,1.001086,1.000000,1.000000,0.984424,0.999124,1.000000,1.000000
482,62,F,1721,1,Básico,1508.11,79.37,176.20,15.87,463.59,...,0.0,0.0,1.000000,1.000000,0.998279,1.000915,0.984424,0.999124,1.001907,1.002509
422,63,M,1690,0,Intermediário,430.84,107.71,143.14,61.42,0.00,...,0.0,0.0,1.005672,1.001086,0.998279,1.000915,0.984424,0.999124,1.000000,1.000000
778,40,F,467,0,Avançado,364.08,91.02,189.14,9.01,0.00,...,0.0,0.0,1.005672,1.001086,0.998279,1.000915,1.000000,1.000000,1.001907,1.002509


## **Normalização e Padronização**

Pode ser usado tanto a normalização como a padronização. Vou optar pela **padronização**, pois pode favorecer quando uma distribuição for normal e não prejudica se não for.

In [34]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Instanciando o scaler
scaler = StandardScaler()

# Selecionando colunas numéricas
numeric_cols = abt02.select_dtypes(include=['float64','int32']).columns

# Aplicando a normalização
abt02[numeric_cols] = scaler.fit_transform(abt02[numeric_cols])
abt02.shape

(700, 49)

In [35]:
abt02.head()

,Idade,GENERO,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,Plano,VL_TOT_CONSUMO,VL_MED_CONSUMO,VL_MAX_CONSUMO,VL_MIN_CONSUMO,VL_TOT_CONS_ESPORTES,...,VL_MAX_CONS_Eletronicos,VL_MIN_CONS_Eletronicos,VL_RAZ_MED_U3M_U6M_CONS_ESP,VL_RAZ_MED_U6M_U12M_CONS_ESP,VL_RAZ_MED_U3M_U6M_CONS_ALI,VL_RAZ_MED_U6M_U12M_CONS_ALI,VL_RAZ_MED_U3M_U6M_CONS_Livros,VL_RAZ_MED_U6M_U12M_CONS_Livros,VL_RAZ_MED_U3M_U6M_CONS_Roupas,VL_RAZ_MED_U6M_U12M_CONS_Roupas
541,1.583371,F,0.135225,-1.049811,Intermediário,-1.245930,2.150143,-0.548968,3.586516,-1.122017,...,0.0,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000
440,0.785320,F,1.646397,0.952553,Intermediário,-0.060821,-0.875874,0.515401,-0.627463,-1.032620,...,0.0,0.0,0.000000,1.653675e-15,0.024635,-0.005813,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897
482,1.250849,F,1.537903,0.952553,Básico,0.808657,-1.045152,0.037201,-0.491992,1.314406,...,0.0,0.0,-0.058542,-8.084448e-03,0.000000,0.000000,1.227625e-15,-6.866812e-16,2.230551e-15,0.000000
422,1.317354,M,1.477843,-1.049811,Intermediário,-1.030785,0.194473,-1.064490,0.933108,-1.122017,...,0.0,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897
778,-0.212243,F,-0.891597,-1.049811,Avançado,-1.144778,-0.535567,0.468414,-0.706618,-1.122017,...,0.0,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000


### **Aplicando OneHotEncoder/LabelEncoder**

**Tratamento das variáveis categóricas - Dummys**



**OneHotEncoder**

OneHotEncoder é aplicado nas variáveis categoricas com cardinalidade menor ou igual a 2 ( valor de corte estipulado ). Baixa cardinalidade.

In [49]:
cut_off = 2

df_categ_onehot = metadata_df[(metadata_df['cardinalidade'] <= cut_off) & (metadata_df['tipo'] == 'object')]
lista_onehot = list(df_categ_onehot.nome_variavel.values)
print('Lista de vars para OneHot Encoding: ',lista_onehot)

# Selecionar colunas categóricas
categorical_cols = abt02.select_dtypes(include=['object']).columns

cols_to_onehot = [col for col in categorical_cols if abt02[col].nunique() < cut_off]

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_cols = encoder.fit_transform(abt02[cols_to_onehot])

# Obtendo os nomes das colunas resultantes
encoded_cols_names = encoder.get_feature_names_out(cols_to_onehot)

# Criando um DataFrame com as colunas codificadas, preservando os índices originais
encoded_df = pd.DataFrame(encoded_cols,columns = encoded_cols_names, index = abt02.index)

# Concatenando o DataFrame criando com o nosso
abt02 = pd.concat([abt02.drop(columns=cols_to_onehot), encoded_df], axis=1)

abt02

Lista de vars para OneHot Encoding:  ['GENERO']


,Idade,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,Plano,VL_TOT_CONSUMO,VL_MED_CONSUMO,VL_MAX_CONSUMO,VL_MIN_CONSUMO,VL_TOT_CONS_ESPORTES,VL_MED_CONS_ESPORTES,...,VL_MIN_CONS_Eletronicos,VL_RAZ_MED_U3M_U6M_CONS_ESP,VL_RAZ_MED_U6M_U12M_CONS_ESP,VL_RAZ_MED_U3M_U6M_CONS_ALI,VL_RAZ_MED_U6M_U12M_CONS_ALI,VL_RAZ_MED_U3M_U6M_CONS_Livros,VL_RAZ_MED_U6M_U12M_CONS_Livros,VL_RAZ_MED_U3M_U6M_CONS_Roupas,VL_RAZ_MED_U6M_U12M_CONS_Roupas,GENERO_M
541,1.583371,0.135225,-1.049811,Intermediário,-1.245930,2.150143,-0.548968,3.586516,-1.122017,3.789065e-16,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000,0.0
440,0.785320,1.646397,0.952553,Intermediário,-0.060821,-0.875874,0.515401,-0.627463,-1.032620,-2.245567e+00,...,0.0,0.000000,1.653675e-15,0.024635,-0.005813,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897,0.0
482,1.250849,1.537903,0.952553,Básico,0.808657,-1.045152,0.037201,-0.491992,1.314406,-6.388440e-01,...,0.0,-0.058542,-8.084448e-03,0.000000,0.000000,1.227625e-15,-6.866812e-16,2.230551e-15,0.000000,0.0
422,1.317354,1.477843,-1.049811,Intermediário,-1.030785,0.194473,-1.064490,0.933108,-1.122017,3.789065e-16,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897,1.0
778,-0.212243,-0.891597,-1.049811,Avançado,-1.144778,-0.535567,0.468414,-0.706618,-1.122017,3.789065e-16,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,-0.012731,-1.414695,-1.049811,Básico,0.301802,-0.441524,0.286798,-0.002984,-0.952000,-1.836554e+00,...,0.0,0.000000,1.653675e-15,0.000000,-1.021964,1.227625e-15,-6.866812e-16,-1.915655e-02,1.836538,1.0
270,-0.345252,-1.480567,-1.049811,Básico,0.415043,-1.024157,0.237812,-0.770755,-0.577699,6.239751e-02,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,-1.915655e-02,-0.873790,1.0
860,-1.542328,-0.279379,0.952553,Intermediário,2.039784,0.614826,0.403432,-0.541425,0.998705,8.873562e-01,...,0.0,0.000000,-8.084448e-03,0.000000,0.000000,1.227625e-15,5.417584e-03,2.230551e-15,0.000000,1.0
435,1.051337,-1.589061,0.952553,Básico,-0.142149,0.106554,0.301461,-0.069312,-0.747191,-1.748299e+00,...,0.0,-0.058542,1.332464e+00,0.000000,0.000000,6.364410e+00,-9.223436e-01,-1.915655e-02,-0.017897,0.0


**Label Encoder**

O LabelEncoder é aplicado para colunas categóricas cuja cardinalidade exceda o valor de corte ( 2 ). Alta cardinalidade.

In [50]:
# Selecionar colunas categóricas
categorical_cols = abt02.select_dtypes(include=['object']).columns

cols_to_onehot = [col for col in categorical_cols if abt02[col].nunique() >= cut_off]

cols_to_onehot


['Plano']

In [54]:
# Aplicando LabelEncoder nas colunas desejadas
for col in categorical_cols:
    encoder = LabelEncoder()
    abt02[col] = encoder.fit_transform(abt02[col])

abt02.head()


,Idade,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,Plano,VL_TOT_CONSUMO,VL_MED_CONSUMO,VL_MAX_CONSUMO,VL_MIN_CONSUMO,VL_TOT_CONS_ESPORTES,VL_MED_CONS_ESPORTES,...,VL_MIN_CONS_Eletronicos,VL_RAZ_MED_U3M_U6M_CONS_ESP,VL_RAZ_MED_U6M_U12M_CONS_ESP,VL_RAZ_MED_U3M_U6M_CONS_ALI,VL_RAZ_MED_U6M_U12M_CONS_ALI,VL_RAZ_MED_U3M_U6M_CONS_Livros,VL_RAZ_MED_U6M_U12M_CONS_Livros,VL_RAZ_MED_U3M_U6M_CONS_Roupas,VL_RAZ_MED_U6M_U12M_CONS_Roupas,GENERO_M
541,1.583371,0.135225,-1.049811,2,-1.245930,2.150143,-0.548968,3.586516,-1.122017,3.789065e-16,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000,0.0
440,0.785320,1.646397,0.952553,2,-0.060821,-0.875874,0.515401,-0.627463,-1.032620,-2.245567e+00,...,0.0,0.000000,1.653675e-15,0.024635,-0.005813,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897,0.0
482,1.250849,1.537903,0.952553,1,0.808657,-1.045152,0.037201,-0.491992,1.314406,-6.388440e-01,...,0.0,-0.058542,-8.084448e-03,0.000000,0.000000,1.227625e-15,-6.866812e-16,2.230551e-15,0.000000,0.0
422,1.317354,1.477843,-1.049811,2,-1.030785,0.194473,-1.064490,0.933108,-1.122017,3.789065e-16,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897,1.0
778,-0.212243,-0.891597,-1.049811,0,-1.144778,-0.535567,0.468414,-0.706618,-1.122017,3.789065e-16,...,0.0,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000,0.0


## **Compondo a ABT tratada**

Trazer a variável target novamente para compor a tabela.

In [55]:
# Inner join usando o índice
abt_model = pd.merge(abt02, abt00[['Churn']], left_index=True, right_index=True, how='inner')
abt_model.shape

(700, 50)

In [56]:
abt_model.head()

,Idade,QT_DIAS_DESDE_INSCR,FL_USOU_SUPORTE,Plano,VL_TOT_CONSUMO,VL_MED_CONSUMO,VL_MAX_CONSUMO,VL_MIN_CONSUMO,VL_TOT_CONS_ESPORTES,VL_MED_CONS_ESPORTES,...,VL_RAZ_MED_U3M_U6M_CONS_ESP,VL_RAZ_MED_U6M_U12M_CONS_ESP,VL_RAZ_MED_U3M_U6M_CONS_ALI,VL_RAZ_MED_U6M_U12M_CONS_ALI,VL_RAZ_MED_U3M_U6M_CONS_Livros,VL_RAZ_MED_U6M_U12M_CONS_Livros,VL_RAZ_MED_U3M_U6M_CONS_Roupas,VL_RAZ_MED_U6M_U12M_CONS_Roupas,GENERO_M,Churn
541,1.583371,0.135225,-1.049811,2,-1.245930,2.150143,-0.548968,3.586516,-1.122017,3.789065e-16,...,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000,0.0,0
440,0.785320,1.646397,0.952553,2,-0.060821,-0.875874,0.515401,-0.627463,-1.032620,-2.245567e+00,...,0.000000,1.653675e-15,0.024635,-0.005813,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897,0.0,0
482,1.250849,1.537903,0.952553,1,0.808657,-1.045152,0.037201,-0.491992,1.314406,-6.388440e-01,...,-0.058542,-8.084448e-03,0.000000,0.000000,1.227625e-15,-6.866812e-16,2.230551e-15,0.000000,0.0,0
422,1.317354,1.477843,-1.049811,2,-1.030785,0.194473,-1.064490,0.933108,-1.122017,3.789065e-16,...,0.000000,1.653675e-15,0.000000,0.000000,1.227625e-15,-6.866812e-16,-1.915655e-02,-0.017897,1.0,1
778,-0.212243,-0.891597,-1.049811,0,-1.144778,-0.535567,0.468414,-0.706618,-1.122017,3.789065e-16,...,0.000000,1.653675e-15,0.000000,0.000000,1.722314e-01,5.417584e-03,2.230551e-15,0.000000,0.0,1
